# Automated Method - Under Construction

In [3]:
bids_dir = r"C:\Users\Calvin\Dropbox (Partners HealthCare)\resources\datasets\ADNI\neuroimaging\raws\wdrew_reconall"
dockerfile_path = r"C:\Users\Calvin\Documents\Software\Research\nimlab\sbm\dockerfiles\fsl_freesurfer.Dockerfile"

In [6]:
import subprocess
import sys
import os
import sys
import platform
import subprocess

class CalvinSBM:
    """
    CalvinSBM is a class for orchestrating neuroimaging data processing using Docker.
    """

    @staticmethod
    def build_docker_image(dockerfile_path):
        """
        Builds the Docker image from the specified Dockerfile.

        Args:
            dockerfile_path (str): The path to the Dockerfile.
        """
        try:
            print("Building Docker image...")
            subprocess.call(f"docker build -t sbm:latest -f {dockerfile_path} .", shell=True, stdout=None, stderr=None)
            # subprocess.run(["docker", "build", "-t", "cat12:latest", "-f", dockerfile_path, "."], check=True)
            print("Docker image built successfully.")
        except subprocess.CalledProcessError as e:
            print(f"An error occurred while building the Docker image: {e}")
            sys.exit(1)

    @staticmethod
    def convert_path_for_docker(path):
        """
        Converts a Windows path to a Docker-compatible path (if running on Windows).

        Args:
            path (str): The original file system path.

        Returns:
            str: The converted path suitable for Docker volume mounting.
        """
        if platform.system() == "Windows":
            return path.replace("\\", "/") #.replace("C:", "C")
        return path

    @staticmethod
    def run_docker_on_folder(folder_path):
        """
        Runs the Docker container on the specified folder containing NIfTI files.

        Args:
            folder_path (str): The path to the folder containing NIfTI files.
        """
        try:
            docker_path = CalvinSBM.convert_path_for_docker(folder_path)
            print(f"Processing folder with Docker: {folder_path}")
            subprocess.run(["docker", "run", "--rm", "-v", f"{docker_path}:/data", "sbm:latest"], check=True)
            print(f"Finished processing: {folder_path}")
        except subprocess.CalledProcessError as e:
            print(f"An error occurred while processing {folder_path} with Docker: {e}")

    @staticmethod
    def submit_hpc_jobs(data_dir, docker_script_path, job_script_path):
        """
        Submits jobs to an HPC scheduler for processing each subdirectory 
        in the specified data directory.

        Args:
            data_dir (str): The path to the directory containing subdirectories with NIfTI files.
            docker_script_path (str): The path to the Docker run script.
            job_script_path (str): The path to the job submission script for HPC.
        """
        # Ensure the data directory exists
        if not os.path.isdir(data_dir):
            print(f"Data directory does not exist: {data_dir}")
            sys.exit(1)

        # Iterate over all subdirectories and submit a job for each
        for folder_name in os.listdir(data_dir):
            folder_path = os.path.join(data_dir, folder_name)
            if os.path.isdir(folder_path):
                CalvinSBM.submit_hpc_job(folder_path, docker_script_path, job_script_path)

    @staticmethod
    def submit_hpc_job(folder_path, docker_script_path, job_script_path):
        """
        Submits a single job to an HPC scheduler for processing a given folder.

        Args:
            folder_path (str): The path to the folder to process.
            docker_script_path (str): The path to the Docker run script.
            job_script_path (str): The path to the job submission script for HPC.
        """
        try:
            print(f"Submitting job for folder: {folder_path}")
            # Replace the following line with your HPC's job submission command
            subprocess.run([
                "bsub", "-J", "cat12_job", 
                "-o", "/path/to/output.txt", 
                "-e", "/path/to/error.txt", 
                "-q", "normal", 
                "-R", "rusage[mem=6000]", 
                "python", docker_script_path, folder_path
            ], check=True)
        except subprocess.CalledProcessError as e:
            print(f"An error occurred while submitting {folder_path}: {e}")

In [8]:
CalvinSBM.build_docker_image(dockerfile_path)

Building Docker image...
Docker image built successfully.


In [ ]:
science@aan.com

In [ ]:
!docker run -v "C:/Users/Calvin/Dropbox (Partners HealthCare)/resources/datasets/ADNI/neuroimaging/raws/wdrew_reconall":/data sbm:latest /data/list_subdirs.sh

# Manual Method
- Step 1: cd to directory with dockerfile in it. 
- Step 2: > docker build -t lab:latest -f Dockerfile.lab .
- Step 3: wait for completion
- Step 4 (give run apermission to script): chmod +x /path/to/host/directory/run-script.sh
- Step 5: make sure your target BIDS directory has data available offline if you are running from cloud storage
- Step 6 (Run container, but mount data and scripts directory, then ask the container to execute your script.): 
    - **Run Conversion to FS5 Space Gifti**
    <!-- - docker run -v "C:/Users/Calvin/Documents/Software/Research/nimlab/sbm/scripts:/scripts" -v "C:/Users/Calvin/Dropbox (Partners HealthCare)/studies/atrophy_seeds_2023/shared_analysis/niftis_for_elmira/wdrew_reconall:/opt/freesurfer-6.0.1/subjects" sbm:latest /scripts/calvin_cort_thick_to_fs5.sh
    - Test -->
        - docker run -v "C:/Users/Calvin/Documents/Software/Research/nimlab/sbm/scripts:/scripts" -v "C:/Users/Calvin/Dropbox (Partners HealthCare)/resources/datasets/ADNI/neuroimaging/raws/wdrew_reconall:/data" sbm:latest /scripts/calvin_cort_thick_to_fs5.sh
    - **Run Conversion to MNI152 Volume Sppace Nifti**
        - Note: This expects BIDS organization structure.
        - docker run -v "/Users/cu135/Library/CloudStorage/OneDrive-Personal/OneDrive_Documents/Work/Software/Research/nimlab/sbm/scripts:/scripts" -v "/Users/cu135/Dropbox (Partners HealthCare)/studies/atrophy_seeds_2023/shared_analysis/niftis_for_elmira/wmaps/sbm:/data" sbm:latest /scripts/calvin_fs5_to_mni152.sh